In [1]:
%store -r __importRegression

In [2]:
__importRegression

/home/maviator/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from itertools import product
import gc

In [4]:
#sales = pd.read_csv('../input/train.csv')
#all_data = pd.read_csv('all_data.csv')
all_data = pd.read_csv('../input/all_data_1_2_3_12_cat.csv')
#test = pd.read_csv('../input/test_1_2_3_12_cat.csv')
#test = pd.read_csv('../input/test.csv')
#items = pd.read_csv("../input/items.csv")
#categories = pd.read_csv("../input/item_categories.csv")
#print (sales.shape)

In [ ]:
all_data.head()

In [ ]:
test.head()

In [5]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [ ]:
# Add date_block_num for test with value 34 (next month)
test['date_block_num'] = 34

In [6]:
all_data, NAs = reduce_mem_usage(all_data)

('Memory usage of properties dataframe is :', 1582, ' MB')
******************************
('Column: ', 'shop_id')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('uint8'))
******************************
******************************
('Column: ', 'item_id')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('uint16'))
******************************
******************************
('Column: ', 'date_block_num')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('uint8'))
******************************
******************************
('Column: ', 'target')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('int16'))
******************************
******************************
('Column: ', 'target_shop')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('int16'))
******************************
******************************
('Column: ', 'target_item')
('dtype before: ', dtype('int64'))
('dtype after: ', dtype('int16'))
***************************

In [ ]:
test, NAs = reduce_mem_usage(test)

In [7]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

In [ ]:

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [ ]:
all_data, NAS = reduce_mem_usage(all_data)

del grid, gb 
gc.collect();

In [ ]:
all_data.head()

In [ ]:
all_data.to_csv('all_data.csv', index=False)

In [ ]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols))

In [ ]:
cols_to_rename

In [8]:
# lag months: 1 = last month sales; 12 = last year same month sales
shift_range = [1,2,3,12]

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)
    
    # Merge lag feature with train data
    #all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)
    
    # Merge lag feature with test data
    test = pd.merge(test, train_shift, on=index_cols, how='left').fillna(0)
    
del train_shift
gc.collect();

In [ ]:
items = pd.read_csv("../input/items.csv")
#items, NAs = reduce_mem_usage(items)
print items.shape

In [ ]:
items.head()

In [ ]:
# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')

test = pd.merge(test, item_category_mapping, how='left', on='item_id')

In [ ]:
test.head()

In [ ]:
test, NAs = reduce_mem_usage(test)

In [ ]:
test.to_csv('../input/test_1_2_3_12_cat.csv', index=False)

In [ ]:
all_data, NAs = reduce_mem_usage(all_data)

In [ ]:
all_data.to_csv('../input/all_data_1_2_3_12_cat.csv', index=False)

In [9]:
# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
fit_cols.append('item_category_id')
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

print fit_cols
print to_drop_cols

['target_lag_1', 'target_item_lag_1', 'target_shop_lag_1', 'target_lag_2', 'target_item_lag_2', 'target_shop_lag_2', 'target_lag_3', 'target_item_lag_3', 'target_shop_lag_3', 'target_lag_12', 'target_item_lag_12', 'target_shop_lag_12', 'item_category_id']
['target_item', 'target_shop', 'target', 'date_block_num']


## Train / Validation split

In [10]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']

last_block = dates.max()
print('Test `date_block_num` is %d' % last_block)

Test `date_block_num` is 33


In [11]:
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'n_jobs': -1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':0 
              }

lgb = LGBMRegressor(**lgb_params)

In [12]:
# Moving window validation scheme.
# On each iteration, use last month for validation
validation_months = [33, 32, 31, 30, 29]

for last_month in validation_months:
    # Split train and validation data
    dates_train = dates[dates <  last_month]
    dates_test  = dates[dates == last_month]

    X_train = all_data.loc[dates <  last_month].drop(to_drop_cols, axis=1)
    X_test =  all_data.loc[dates == last_month].drop(to_drop_cols, axis=1)

    y_train = all_data.loc[dates <  last_month, 'target'].values
    y_test =  all_data.loc[dates == last_month, 'target'].values
    
    lgb.fit(X_train, y_train)

    pred_train = lgb.predict(X_train)
    pred_test = lgb.predict(X_test)
    
    ## R2 and RMSE score for each validation fold
    print('Month {0:d} Test R-2: {1:f}'.format(last_month, r2_score(y_test, pred_test)))
    print('Month {0:d} Test RMSE {1:f}'.format(last_month, np.sqrt(mean_squared_error(y_test, pred_test))))
    
del dates_train, dates_test, X_train, X_test, y_train, y_test, pred_train, pred_test
gc.collect()

Month 33 Test R-2: 0.113460
Month 33 Test RMSE 5.030523
Month 32 Test R-2: 0.047766
Month 32 Test RMSE 6.682272
Month 31 Test R-2: 0.343891
Month 31 Test RMSE 1.811004
Month 30 Test R-2: 0.417466
Month 30 Test RMSE 1.632446
Month 29 Test R-2: 0.359447
Month 29 Test RMSE 1.863724


447

## Prepare submission

In [ ]:
## Take all train data
X_train_all = all_data.drop(to_drop_cols, axis=1)
y_train_all = all_data.target.values

In [ ]:
X_train_all.target_lag_1.describe()

In [ ]:
test.sort(columns='ID')

In [ ]:
test_id = test.pop('ID')
test.drop('date_block_num', axis=1, inplace=True)

In [ ]:
test.head()

In [ ]:
X_train_all.head()

In [ ]:
lgb.fit(X_train_all, y_train_all)

pred_train_all = lgb.predict(X_train_all)
pred_test = lgb.predict(test)

## R2 and RMSE score for each validation fold
print('Train R-2: {0:f}'.format(r2_score(y_train_all, pred_train_all)))
print('Train RMSE {0:f}'.format(np.sqrt(mean_squared_error(y_train_all, pred_train_all))))

In [ ]:
pred_test.shape

In [ ]:
for i in range(len(pred_test)):
    if pred_test[i] > 20:
        pred_test[i] = 20
    if pred_test[i] < 0:
        pred_test[i] = 20

In [ ]:
test_submit = pd.DataFrame({'ID': test_id, 'item_cnt_month': pred_test})
print test_submit.shape
test_submit.to_csv('lgbm_1_2_3_12_cat.csv', index=False)
test_submit.head()